In [52]:
import csv
import pandas as pd

DATA_PATH = "../data/raw_data/test_dataset.csv"

data = []
with open(DATA_PATH, 'r', encoding='utf-8') as file:
    reader = csv.reader(file, quotechar='"', escapechar='\\')
    for i, row in enumerate(reader):
        if len(row) == 2:
            data.append(row)
        elif len(row) > 2:
            # Gabungkan field tambahan ke tweet
            label = row[0]
            tweet = ','.join(row[1:])
            data.append([label, tweet])
        else:
            print(f"Skipping line {i}: {row}")

# Convert to DataFrame
if data:
    df = pd.DataFrame(data[1:], columns=data[0])
    print(f"Shape: {df.shape}")
    df.head()
else:
    print("No data found")

Skipping line 2731: []
Shape: (2730, 2)


In [53]:
df = df.rename(columns={'label': 'Label', 'teks':'Tweet'})

In [54]:
df

,Label,Tweet
0,Marah,Gue benar-benar jengkel sama orang-orang yang ...
1,Marah,"Dasar pemerintah bangsat, urusan rakyat kecil ..."
2,Marah,Aku benci banget sama diriku sendiri yang sela...
3,Marah,"Anjir, nyebelin banget dah aplikasi gojek lagi..."
4,Marah,Gue muak sama berita-berita hoax yang bikin re...
...,...,...
2725,Sedih,rasanya kaya hati gue tertutup kabut
2726,Sedih,gue sedih tiap denger suara yang dulu hangat
2727,Sedih,gue ngerasa ga ada yang ngerti perasaan gue
2728,Sedih,rasanya seperti luka yang tak kunjung sembuh


In [55]:
# Mengubag nama label menjadi bahasa Indonesia
label_mapping = {
    'Senang': 'Gembira',
    'Cinta': 'Cinta',
    'Takut': 'Takut',
    'Sedih': 'Sedih',
    'Marah': 'Marah'
}
df['Label'] = df['Label'].map(label_mapping)
print(df['Label'].value_counts())

Label
Gembira    1095
Marah       470
Takut       424
Cinta       400
Sedih       341
Name: count, dtype: int64


In [56]:
print(df['Label'].value_counts())

Label
Gembira    1095
Marah       470
Takut       424
Cinta       400
Sedih       341
Name: count, dtype: int64


In [57]:
df = df.loc[:, ~df.columns.str.contains('final_text')]

In [58]:
#remove surprise label from dataset
df = df[df['Label'] != 'surprise'].copy()
print(df['Label'].value_counts())

Label
Gembira    1095
Marah       470
Takut       424
Cinta       400
Sedih       341
Name: count, dtype: int64


In [59]:
#sort df by label
df = df.sort_values(by='Label').reset_index(drop=True)
df.head()

,Label,Tweet
0,Cinta,rasanya kaya hati gue dikenyalkan
1,Cinta,tadi gue ngerasa semua jadi lebih adem karena dia
2,Cinta,gue rindu hal sederhana yang dia lakukan
3,Cinta,gue suka tiap dia ngomong dengan pelan
4,Cinta,rasanya kaya gue ditutupin selimut hangat


In [60]:
# df.to_csv('./data/external_test.csv', index=False)

In [61]:
import numpy as np

N_SAMPLES = 473
LABEL = 'Label'

df_downsampled = (
    df.groupby(LABEL)
    .apply(lambda x: x.sample(
        n=min(len(x), N_SAMPLES),
        random_state=42
    ))
    .reset_index(drop=True)
)

print("Distribusi Label Setelah Downsampling:")
print(df_downsampled[LABEL].value_counts())

Distribusi Label Setelah Downsampling:
Label
Gembira    473
Marah      470
Takut      424
Cinta      400
Sedih      341
Name: count, dtype: int64


/tmp/ipykernel_77207/4123522497.py:8: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sample(


In [62]:
# drop final_text column
df = df_downsampled.loc[:, ~df.columns.str.contains('final_text')]

In [63]:
import pandas as pd
import re
import string

from indoNLP.preprocessing import replace_word_elongation, replace_slang

In [64]:
def cleanDataframe(df):
    print('Missing Values:', df.isnull().sum())
    print('Duplicates:', df.duplicated().sum())

    df_cleaned = df.dropna(subset='Tweet')
    df_cleaned = df_cleaned.drop_duplicates(keep='first')

    print('\nMissing values after cleaning:', df_cleaned.isnull().sum())
    print('Duplicates after cleaning:', df_cleaned.duplicated().sum())
    return df_cleaned


def cleaningText(text):
    # Membersihkan tanda tanda sisa medsos
    text = re.sub(r'\[USERNAME\]', '', text)
    text = re.sub(r'\[URL\]', '', text)
    text = re.sub(r'\[SENSITIVE-NO\]', '', text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)  # Menghapus Mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text)  # Menghapus Hashtag
    text = re.sub(r'RT[\s]', '', text)  # menghapus RT
    text = re.sub(r"http\S+", '', text)  # menghapus link

    # Pembersihan Karakter
    # Mengganti tanda baca dengan spasi (alih-alih menghapus)
    text = text.translate(str.maketrans(
        string.punctuation, ' ' * len(string.punctuation)))
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = text.replace('\n', ' ')  # Mengganti garis baru dengan spasi
    return text


kamusCleaning = pd.read_csv('../data/kamus.csv', encoding='utf-8',
                            delimiter=',', header=None, names=['slang', 'formal'])


cleaningDict = {}
for slang, formal in zip(kamusCleaning.slang, kamusCleaning.formal):
    if pd.notna(slang) and pd.notna(formal):
        slang_lower = str(slang).lower().strip()
        formal_clean = str(formal).strip()
        cleaningDict[slang_lower] = formal_clean

print(f"✅ Loaded {len(cleaningDict)} entries from kamus.csv")


def applyCleaningDict(text):
    if not isinstance(text, str) or not text.strip():
        return text

    # Pattern untuk match whole words (case insensitive)
    words = text.split()
    cleaned_words = []

    for word in words:
        # Clean word (lowercase untuk matching)
        clean_word = word.lower().strip()

        # Check jika word ada di dictionary
        if clean_word in cleaningDict:
            cleaned_words.append(cleaningDict[clean_word])
        else:
            cleaned_words.append(word)

    cleaned_text = ' '.join(cleaned_words)
    return cleaned_text


def normalizeText(text):
    if not isinstance(text, str):
        return ""

    if not text.strip():
        return text.strip()

    # 1. Lowercase pertama
    text = text.lower()

    # 2. Handle character elongation
    text = re.sub(r'(.)\1+', r'\1\1', text)
    if 'replace_word_elongation' in globals():
        text = replace_word_elongation(text)

    # 3. Replace slang words (dari multiple sources)
    if 'replace_slang' in globals():
        text = replace_slang(text)
    text = applyCleaningDict(text)

    # 4. Final cleanup
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    return text

✅ Loaded 1305 entries from kamus.csv


In [65]:
df_cleaned = cleanDataframe(df)

Missing Values: Label    0
Tweet    0
dtype: int64
Duplicates: 158

Missing values after cleaning: Label    0
Tweet    0
dtype: int64
Duplicates after cleaning: 0


In [66]:
df['Tweet'] = df['Tweet'].apply(cleaningText)
df_normalized = df.copy()
df_normalized['Tweet'] = df_normalized['Tweet'].apply(normalizeText)

In [67]:
df_normalized['Label'].value_counts()

Label
Gembira    473
Marah      470
Takut      424
Cinta      400
Sedih      341
Name: count, dtype: int64

In [68]:
df_normalized.to_csv('../data/external_test.csv', index=False)